In [1]:
import os
from os import listdir
from os.path import isfile, join, splitext
import sys
import csv
import re

import numpy as np
import pandas as pd
import geopandas as gpd
import spacy

import usaddress

In [2]:
os.chdir('..')
abs_path = os.getcwd()
print(abs_path)

C:\Users\bchan\OneDrive - UW\CLAD\CLAD_Geospatial


# Parsing with `usaddress` module

## Edge Cases
For initial unit-testing

In [2]:
# Queens, NY address with dashes in street address
queens_ny_address = "89-22 197th St Unit 2, Hollis, NY 11423"
usaddress.tag(queens_ny_address)

(OrderedDict([('AddressNumber', '89-22'),
              ('StreetName', '197th'),
              ('StreetNamePostType', 'St'),
              ('OccupancyType', 'Unit'),
              ('OccupancyIdentifier', '2'),
              ('PlaceName', 'Hollis'),
              ('StateName', 'NY'),
              ('ZipCode', '11423')]),
 'Street Address')

In [3]:
# Southwest address with Spanish names
NM_address = "220 Camino Tres SW, Albuquerque, NM 87105"
usaddress.tag(NM_address)

(OrderedDict([('AddressNumber', '220'),
              ('StreetNamePreType', 'Camino'),
              ('StreetName', 'Tres'),
              ('StreetNamePostDirectional', 'SW'),
              ('PlaceName', 'Albuquerque'),
              ('StateName', 'NM'),
              ('ZipCode', '87105')]),
 'Street Address')

In [4]:
# grid system with no street type
UT_address = "921 3385 S, Millcreek, UT 84106"
usaddress.tag(UT_address)

(OrderedDict([('AddressNumber', '921'),
              ('StreetName', '3385'),
              ('StreetNamePostDirectional', 'S'),
              ('PlaceName', 'Millcreek'),
              ('StateName', 'UT'),
              ('ZipCode', '84106')]),
 'Street Address')

In [5]:
# highway_address = "1550 United States Highway 20 South, Worland, WY 82401"
highway_28_address = "1550 US Highway 20 S, Worland, WY 82401"
usaddress.tag(highway_28_address)

(OrderedDict([('AddressNumber', '1550'),
              ('StreetNamePreType', 'US Highway'),
              ('StreetName', '20'),
              ('StreetNamePostDirectional', 'S'),
              ('PlaceName', 'Worland'),
              ('StateName', 'WY'),
              ('ZipCode', '82401')]),
 'Street Address')

## Nominatim Outputs
* `usaddress` cannot parse Nominatim display addresses correctly
* Nominatim display name convention: "amenity", "house_number", "road", "neighbourhood", "city" OR "town", "county", "state", "postcode", "country" 

In [3]:
nominatim_output_path = os.path.join(abs_path, 'output', 'nominatim_address_for_geocoders_output_Aug13 1.xlsx')
nominatim_output = pd.read_excel(nominatim_output_path, sheet_name='nominatim_responses', engine="openpyxl", header=0)
nominatim_output

,geo_lat,geo_long,geo_address,geo_display_name,geo_class,geo_type,id,amenity,house_number,road,...,municipality,residential,office,shop,man_made,region,industrial,quarter,retail,leisure
0,43.652336,-70.264556,"{'amenity': ""Saint Luke's Cathedral"", 'house_n...","Saint Luke's Cathedral, 143, State Street, Par...",amenity,place_of_worship,0,Saint Luke's Cathedral,143.0,State Street,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43.649002,-70.541248,"{'house_number': '938', 'road': 'Long Plains R...","938, Long Plains Road, Buxton Center, Buxton, ...",building,church,1,NaN,938.0,Long Plains Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43.461360,-70.699863,"{'house_number': '44', 'road': 'Mouse Lane', '...","44, Mouse Lane, Alfred Mills, Alfred, York Cou...",place,house,2,NaN,44.0,Mouse Lane,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,44.048470,-69.503421,"{'house_number': '672', 'road': 'Main Street',...","672, Main Street, Damariscotta, Lincoln County...",place,house,3,NaN,672.0,Main Street,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43.519612,-70.719167,"{'house_number': '1176', 'road': 'Main Street'...","1176, Main Street, Waterboro, York County, Mai...",building,church,4,NaN,1176.0,Main Street,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,41.817812,-103.664446,"{'road': '13th Street', 'town': 'Gering', 'cou...","13th Street, Gering, Scotts Bluff County, Nebr...",highway,residential,524,NaN,NaN,13th Street,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692,34.115167,-81.712531,"{'road': 'Hollywood Road', 'hamlet': 'Higgins'...","Hollywood Road, Higgins, Saluda County, South ...",highway,tertiary,525,NaN,NaN,Hollywood Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,34.089176,-81.669372,"{'road': 'Hollywood Road', 'county': 'Saluda C...","Hollywood Road, Saluda County, South Carolina,...",highway,tertiary,525,NaN,NaN,Hollywood Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
694,34.130325,-81.760500,"{'road': 'Hollywood Road', 'hamlet': 'Higgins'...","Hollywood Road, Higgins, Saluda County, South ...",highway,unclassified,525,NaN,NaN,Hollywood Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
failed_addresses = nominatim_output.loc[nominatim_output.house_number.isna()]

In [24]:
nominatim_output.amenity.notna().sum()

160

In [27]:
nominatim_output.columns

Index(['geo_lat', 'geo_long', 'geo_address', 'geo_display_name', 'geo_class',
       'geo_type', 'id', 'amenity', 'house_number', 'road', 'neighbourhood',
       'city', 'county', 'state', 'ISO3166-2-lvl4', 'postcode', 'country',
       'country_code', 'town', 'village', 'hamlet', 'suburb', 'city_district',
       'building', 'municipality', 'residential', 'office', 'shop', 'man_made',
       'region', 'industrial', 'quarter', 'retail', 'leisure'],
      dtype='object')

In [17]:
address_list = pd.read_excel(nominatim_output_path, sheet_name='address_list', engine="openpyxl", header=0)

In [26]:
address_list.merge(failed_addresses, on='id', how='right').tail(10).address.tolist()

['242,  Church Street , Naytahwaush, Mahnomen County, Minnesota, 56566, United States',
 '1402,  Wyoming Way , Madison, Dane County, Wisconsin, 53704, United States',
 '100,West  Mispillion Street , Harrington, Kent County, Delaware, 19952, United States',
 '2600,  Garland  , Detroit, Wayne County, Michigan, 48214, United States',
 '2600,  Garland  , Detroit, Wayne County, Michigan, 48214, United States',
 '1725,  13th Street , Gering, Scotts Bluff County, Nebraska, 69341, United States',
 '1261,  Hollywood Road , Saluda, Saluda County, South Carolina, 29138, United States',
 '1261,  Hollywood Road , Saluda, Saluda County, South Carolina, 29138, United States',
 '1261,  Hollywood Road , Saluda, Saluda County, South Carolina, 29138, United States',
 '1261,  Hollywood Road , Saluda, Saluda County, South Carolina, 29138, United States']

In [4]:
print(nominatim_output.geo_address[0])
nominatim_address = nominatim_output.geo_display_name[0]
print(nominatim_address)
usaddress.tag(nominatim_address)

{'amenity': "Saint Luke's Cathedral", 'house_number': '143', 'road': 'State Street', 'neighbourhood': 'Parkside', 'city': 'Portland', 'county': 'Cumberland County', 'state': 'Maine', 'ISO3166-2-lvl4': 'US-ME', 'postcode': '04101', 'country': 'United States', 'country_code': 'us'}
Saint Luke's Cathedral, 143, State Street, Parkside, Portland, Cumberland County, Maine, 04101, United States


(OrderedDict([('BuildingName',
               "Saint Luke's Cathedral, 143, State Street, Parkside, Portland, Cumberland County, Maine"),
              ('AddressNumber', '04101'),
              ('StreetName', 'United States')]),
 'Street Address')

In [8]:
print(eval(nominatim_output.geo_address[691]))
nominatim_address = nominatim_output.geo_display_name[691]
print(nominatim_address)
usaddress.tag(nominatim_address)

{'road': '13th Street', 'town': 'Gering', 'county': 'Scotts Bluff County', 'state': 'Nebraska', 'ISO3166-2-lvl4': 'US-NE', 'postcode': '69341', 'country': 'United States', 'country_code': 'us'}
13th Street, Gering, Scotts Bluff County, Nebraska, 69341, United States


(OrderedDict([('StreetName', '13th'),
              ('StreetNamePostType', 'Street'),
              ('PlaceName', 'Gering, Scotts Bluff County'),
              ('StateName', 'Nebraska'),
              ('ZipCode', '69341'),
              ('CountryName', 'United States')]),
 'Ambiguous')

In [6]:
print(nominatim_output.geo_address[695])
nominatim_address = nominatim_output.geo_display_name[695]
print(nominatim_address)
usaddress.tag(nominatim_address)

{'road': 'Hollywood Road', 'county': 'Saluda County', 'state': 'South Carolina', 'ISO3166-2-lvl4': 'US-SC', 'country': 'United States', 'country_code': 'us'}
Hollywood Road, Saluda County, South Carolina, United States


(OrderedDict([('StreetName', 'Hollywood'),
              ('StreetNamePostType', 'Road'),
              ('Recipient', 'Saluda County, South Carolina, United States')]),
 'Ambiguous')

# Simulated Residential History Data
Set the staging for parsing

In [3]:
HIFLD_path = os.path.join(abs_path, 'output/HIFLD/centroids')

csv_files = [f for f in listdir(HIFLD_path) if isfile(join(HIFLD_path, f)) and f.endswith(".csv")]
csv_paths = [join(HIFLD_path, a) for a in csv_files]
csv_paths

['C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\output/HIFLD/centroids\\centroids_40k_OMOP.csv',
 'C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\output/HIFLD/centroids\\centroids_432k.csv',
 'C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\output/HIFLD/centroids\\centroids_70k.csv']

In [15]:
test_list = []
for path in csv_paths:
    df = pd.read_csv(path)
    test_list.append(df)

In [4]:
# import cleaned HIFLD addresses
# link = os.path.join(abs_path, 'output', 'HIFLD', 'centroids', 'centroids_70k.csv') 
temp = pd.read_csv(csv_paths[1])
temp['Full_Address'] = temp.Full_Address

# temp = pd.read_csv(link) 
# temp = pd.concat(test_list)
temp

,Full_Address,Place_type,source_centroid,source_lon,source_lat
0,"523 E BROADWAY, SOUTH BOSTON, MA 02127",AllPlacesOfWorship,POINT (-71.04352199999346 42.33547200002667),-71.043522,42.335472
1,"454 ESSEX ST, LAWRENCE, MA 01840",AllPlacesOfWorship,POINT (-71.16493999969576 42.70621300000711),-71.164940,42.706213
2,"569 BROADWAY, NEWARK, NJ 07104",AllPlacesOfWorship,POINT (-74.16282099988595 40.76993499987205),-74.162821,40.769935
3,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",AllPlacesOfWorship,POINT (-78.74781599997843 42.79853499984358),-78.747816,42.798535
4,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",AllPlacesOfWorship,POINT (-70.11377699970232 44.42860999977688),-70.113777,44.428610
...,...,...,...,...,...
432846,"115 STEWARTS FERRY PIKE, NASHVILLE, TN 37214",PublicSchools,POINT (-86.65375093130474 36.16762100414446),-86.653751,36.167621
432847,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",PublicSchools,POINT (-90.01485112798842 35.1654996828778),-90.014851,35.165500
432848,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",PublicSchools,POINT (-90.01468838380168 35.16544009791454),-90.014688,35.165440
432849,"2610 CINEMA DR, MARYVILLE, TN 37804",PublicSchools,POINT (-83.94173034130181 35.79683260169423),-83.941730,35.796833


In [20]:
# import cleaned HIFLD addresses
link = os.path.join(abs_path, 'output', 'sample_spatial_join_OMOP_clean.csv')

temp = pd.read_csv(link)
temp

C:\Users\bchan\AppData\Local\Temp\ipykernel_27000\614527325.py:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(link)


,Full_Addre,Place_type,source_lon,source_lat,geometry,index_right,OBJECTID,GEOID_1,NAME_1,NAMELSAD_1,...,Shape_Leng,Shape_Le_1,Shape_Area,Tribal,address_1,address_2,city,state,zip,STATE
0,"523 E BROADWAY, SOUTH BOSTON, MA 02127",AllPlacesOfWorship,-71.043522,42.335472,POINT (-71.04352199999346 42.33547200002667),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,523 E BROADWAY,NaN,SOUTH BOSTON,MA,2127,MA
1,"454 ESSEX ST, LAWRENCE, MA 01840",AllPlacesOfWorship,-71.164940,42.706213,POINT (-71.16493999969576 42.70621300000711),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,454 ESSEX ST,NaN,LAWRENCE,MA,1840,MA
2,"569 BROADWAY, NEWARK, NJ 07104",AllPlacesOfWorship,-74.162821,40.769935,POINT (-74.16282099988595 40.76993499987205),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,569 BROADWAY,NaN,NEWARK,NJ,7104,NJ
3,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",AllPlacesOfWorship,-78.747816,42.798535,POINT (-78.74781599997843 42.79853499984358),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,3210 SOUTHWESTERN BLVD,NaN,ORCHARD PARK,NY,14127,NY
4,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",AllPlacesOfWorship,-70.113777,44.428610,POINT (-70.11377699970232 44.42860999977688),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,431 CAMPGROUND RD,NaN,LIVERMORE FLS,ME,4254,ME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104097,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",UrgentCareFacs,-84.095174,34.041727,POINT (-84.09517421962246 34.04172694900986),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,1300 PEACHTREE INDUSTRIAL BOULEVARD,NaN,SUWANEE,GA,30024,GA
104098,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",UrgentCareFacs,-84.101318,33.966797,POINT (-84.10131848955332 33.96679709299809),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2660 SATELLITE BOULEVARD NORTHWEST,NaN,DULUTH,GA,30096,GA
104099,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",UrgentCareFacs,-83.902215,34.068832,POINT (-83.90221543184028 34.06883234729177),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,3685 BRASELTON HIGHWAY,NaN,DACULA,GA,30019,GA
104100,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",UrgentCareFacs,-84.494106,37.996508,POINT (-84.49410603994914 37.99650802456039),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,1055 DOVE RUN ROAD,NaN,LEXINGTON,KY,40502,KY


In [5]:
# USPS Publication 28 Standard dictionary for `usaddress`
Pub28_usaddress_template = {
   'Recipient': 'recipient',
   'AddressNumber': 'address1',
   'AddressNumberPrefix': 'address1',
   'AddressNumberSuffix': 'address1',
   'StreetName': 'address1',
   'StreetNamePreDirectional': 'address1',
   'StreetNamePreModifier': 'address1',
   'StreetNamePreType': 'address1',
   'StreetNamePostDirectional': 'address1',
   'StreetNamePostModifier': 'address1',
   'StreetNamePostType': 'address1',
   'CornerOf': 'address1',
   'IntersectionSeparator': 'address1',
   'LandmarkName': 'address1',
   'USPSBoxGroupID': 'address1',
   'USPSBoxGroupType': 'address1',
   'USPSBoxID': 'address1',
   'USPSBoxType': 'address1',
   'BuildingName': 'address2',
   'OccupancyType': 'address2',
   'OccupancyIdentifier': 'address2',
   'SubaddressIdentifier': 'address2',
   'SubaddressType': 'address2',
   'PlaceName': 'city',
   'StateName': 'state',
   'ZipCode': 'zip_code',
}

In [7]:
# 1) identify unique address strings for location records
temp_drop = temp.drop_duplicates(subset='Full_Address')

# 2) set up OMOP placeholder table
OMOP_location = pd.DataFrame(columns=['Location_id','address_1','address_2','city','state','zip','county',
                                      'location_source_value','latitude','longitude'])
                             
OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude


In [8]:
# 3) assign address_strings to OMOP_location table with location_source_value series
OMOP_location['location_source_value'] = temp_drop.Full_Address

# 4) set unique location ID for each address
OMOP_location['Location_id'] = OMOP_location.index+1

# 5) assign the source latitutde and longitude for each address
OMOP_location.latitude = temp_drop.source_lat
OMOP_location.longitude = temp_drop.source_lon
OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude
0,1,NaN,NaN,NaN,NaN,NaN,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522
1,2,NaN,NaN,NaN,NaN,NaN,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940
2,3,NaN,NaN,NaN,NaN,NaN,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821
3,4,NaN,NaN,NaN,NaN,NaN,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816
4,5,NaN,NaN,NaN,NaN,NaN,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777
...,...,...,...,...,...,...,...,...,...,...
432841,432842,NaN,NaN,NaN,NaN,NaN,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458
432842,432843,NaN,NaN,NaN,NaN,NaN,NaN,"6700 E 41ST ST, SIOUX FALLS, SD 57110",43.516499,-96.642310
432843,432844,NaN,NaN,NaN,NaN,NaN,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432
432847,432848,NaN,NaN,NaN,NaN,NaN,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851


## Parse OMOP Components

**USPS Publication 28 Standard**
  * address_1
  * address_2
  * city
  * state
  * zip
  * county
  * location_source_value
  * latitude
  * longitude

In [ ]:
n = 10
list_df = np.array_split(OMOP_location, math.ceil(len(df) / n))

In [9]:
%%time
# 6) process address string through usaddress parser with USPS Pub28 template
repo = pd.DataFrame()

# iterate over OMOP_location
for ind, each in OMOP_location.loc[:,['location_source_value']].drop_duplicates().iterrows():

    # try parsing with usaddress parser using the USPS Pub28 template
    try:
        obj = usaddress.tag(each.location_source_value, tag_mapping=Pub28_usaddress_template)
        
        # staging
        tmp = pd.DataFrame(obj[0], columns=obj[0].keys(), index=[ind])
        tmp['Address_type'] = obj[1]
        
        # development
        OMOP_location.loc[ind, 'address_1'] = tmp['address1'].values[0]
        OMOP_location.loc[ind, 'city'] = tmp['city'].values[0]
        OMOP_location.loc[ind, 'state'] = tmp['state'].values[0]
        OMOP_location.loc[ind, 'zip'] = tmp['zip_code'].values[0]
        OMOP_location.loc[ind, 'address_type']=tmp['Address_type'].values[0]

        address_2 = tmp['address2'].values[0]
        if len(address_2) >= 3:
            OMOP_location.loc[ind, 'address_2'] = address_2
        else:
            OMOP_location.loc[ind, 'address_2'] = np.NaN

        repo = repo.append(tmp)
    
    except:
        # print(ind, each.location_source_value)
        pass


CPU times: total: 59min 15s
Wall time: 2h 19min 3s


In [10]:
OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type
0,1,523 E BROADWAY,NaN,SOUTH BOSTON,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address
1,2,454 ESSEX ST,NaN,LAWRENCE,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address
2,3,569 BROADWAY,NaN,NEWARK,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address
3,4,3210 SOUTHWESTERN BLVD,NaN,ORCHARD PARK,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address
4,5,431 CAMPGROUND RD,NaN,LIVERMORE FLS,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address
...,...,...,...,...,...,...,...,...,...,...,...
432841,432842,201 E 38TH ST,NaN,SIOUX FALLS,SD,57105,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458,Street Address
432842,432843,6700 E 41ST ST,NaN,SIOUX FALLS,SD,57110,NaN,"6700 E 41ST ST, SIOUX FALLS, SD 57110",43.516499,-96.642310,Street Address
432843,432844,216 10TH ST SE,NaN,WATERTOWN,SD,57201,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432,Street Address
432847,432848,1250 VOLLINTINE AVE,NaN,MEMPHIS,TN,38107,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851,Street Address


In [11]:
# check that address_2 parsed correctly
OMOP_location.loc[OMOP_location.address_2.notna()]

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type
6,7,337 STATE STREET,SUITE 3,AUGUSTA,ME,04330,NaN,"337 STATE STREET SUITE 3, AUGUSTA, ME 04330",44.297293,-69.783541,Street Address
79,80,75 MORTON VILLAGE DR,APT 408,MATTAPAN,MA,02126,NaN,"75 MORTON VILLAGE DR APT 408, MATTAPAN, MA 02126",42.280395,-71.085149,Street Address
81,82,200 HANCOCK ST,APT 904,BANGOR,ME,04401,NaN,"200 HANCOCK ST APT 904, BANGOR, ME 04401",44.802406,-68.762050,Street Address
130,131,430 GRANDVIEW AVE,APT 17,BANGOR,ME,04401,NaN,"430 GRANDVIEW AVE APT 17, BANGOR, ME 04401",44.827785,-68.781941,Street Address
187,188,33 GLENWOOD DR,APT 2,BANGOR,ME,04401,NaN,"33 GLENWOOD DR APT 2, BANGOR, ME 04401",44.823394,-68.795611,Street Address
...,...,...,...,...,...,...,...,...,...,...,...
432639,432640,600 STEELHEAD WAY,# 164,BOISE,ID,83704,NaN,"600 STEELHEAD WAY #164, BOISE, ID 83704",43.610421,-116.289425,Street Address
432661,432662,1 TAYLOR ST,RM 101,CHESTER,IL,62233,NaN,"1 TAYLOR ST RM 101, CHESTER, IL 62233",37.903757,-89.828245,Street Address
432675,432676,311 E MAIN ST,STE 632,GALESBURG,IL,61401,NaN,"311 E MAIN ST STE 632, GALESBURG, IL 61401",40.947703,-90.366041,Street Address
432826,432827,425 SUMMIT TERRACE COURT,BUILDING 2,COLUMBIA,SC,29229,NaN,"425 SUMMIT TERRACE COURT BUILDING 2, COLUMBIA,...",34.143948,-80.886625,Street Address


# Post-Hoc Processing
1) Replace full name states to abbreviation
2) Capitalize only the first letter of each parsed component

In [17]:
# dictionary for state full name to abbreviation
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

In [13]:
# make copy of OMOP_location
OMOP_location_copy = OMOP_location

# eliminate whitespaces from end of `state`
OMOP_location_copy['state'] = OMOP_location_copy.state.apply(lambda x: str(x).strip())

# replace full state names to abbreviations
OMOP_location_copy['state_abbr'] = OMOP_location_copy.state.apply(lambda x: multipleReplace(str(x), us_state_to_abbrev))
OMOP_location_copy

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
0,1,523 E BROADWAY,NaN,SOUTH BOSTON,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA
1,2,454 ESSEX ST,NaN,LAWRENCE,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA
2,3,569 BROADWAY,NaN,NEWARK,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ
3,4,3210 SOUTHWESTERN BLVD,NaN,ORCHARD PARK,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY
4,5,431 CAMPGROUND RD,NaN,LIVERMORE FLS,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME
...,...,...,...,...,...,...,...,...,...,...,...,...
432841,432842,201 E 38TH ST,NaN,SIOUX FALLS,SD,57105,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458,Street Address,SD
432842,432843,6700 E 41ST ST,NaN,SIOUX FALLS,SD,57110,NaN,"6700 E 41ST ST, SIOUX FALLS, SD 57110",43.516499,-96.642310,Street Address,SD
432843,432844,216 10TH ST SE,NaN,WATERTOWN,SD,57201,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432,Street Address,SD
432847,432848,1250 VOLLINTINE AVE,NaN,MEMPHIS,TN,38107,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851,Street Address,TN


In [14]:
OMOP_location_copy.state_abbr.value_counts()

state_abbr
CA                  45368
TX                  31952
FL                  23874
NY                  21616
IL                  15091
                    ...  
CA NOT AVAILABLE        1
FL NOT AVAILABLE        1
MD NOT AVAILABLE        1
AP                      1
CA, CA                  1
Name: count, Length: 69, dtype: int64

In [15]:
OMOP_location_copy.loc[OMOP_location_copy.state_abbr == "Dakota"]

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
18031,18032,101 N. Phillips Avenue,NaN,"Sioux Falls, South",Dakota,57104,NaN,"101 N. Phillips Avenue, Sioux Falls, South Dak...",31.002757,-86.326164,Street Address,Dakota
18261,18262,220 Main Street,NaN,"Hoven, South",Dakota,57450,NaN,"220 Main Street, Hoven, South Dakota 57450",45.459900,-98.474698,Street Address,Dakota
18262,18263,"7 East Highway 12, Groton",NaN,South,Dakota,57445,NaN,"7 East Highway 12, Groton, South Dakota 57445",45.457707,-98.102382,Street Address,Dakota
18263,18264,1600 Main Street,NaN,"Tyndall, South",Dakota,57066,NaN,"1600 Main Street, Tyndall, South Dakota 57066",42.991351,-97.862675,Street Address,Dakota
18265,18266,803 Main Street,NaN,"Timber Lake, South",Dakota,57656,NaN,"803 Main Street, Timber Lake, South Dakota 57656",45.428378,-101.073880,Street Address,Dakota
18266,18267,420 South Pierre Street,NaN,"Pierre, South",Dakota,57501,NaN,"420 South Pierre Street, Pierre, South Dakota ...",43.254257,-98.695554,Street Address,Dakota
18268,18269,124 Elk Street,NaN,"Elkton, South",Dakota,57026,NaN,"124 Elk Street, Elkton, South Dakota 57026",44.318317,-96.920602,Street Address,Dakota
18270,18271,101 South Main Street,NaN,"Plankinton, South",Dakota,57368,NaN,"101 South Main Street, Plankinton, South Dakot...",43.714273,-98.485217,Street Address,Dakota
18272,18273,1204 Yellowstone Street,NaN,"Fort Pierre, South",Dakota,57532,NaN,"1204 Yellowstone Street, Fort Pierre, South Da...",44.385579,-100.382734,Street Address,Dakota
18273,18274,702 7th Street,NaN,"Eureka, South",Dakota,57437,NaN,"702 7th Street, Eureka, South Dakota 57437",45.769481,-99.619770,Street Address,Dakota


In [16]:
# capitalize only first letter
OMOP_location_copy['address_1'] = OMOP_location_copy.address_1.apply(lambda x: str(x).strip().title())
OMOP_location_copy['address_2'] = OMOP_location_copy.address_2.apply(lambda x: str(x).strip().title() if not np.NaN else x)
OMOP_location_copy['city'] = OMOP_location_copy.city.apply(lambda x: str(x).strip().title())

OMOP_location_copy

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME
...,...,...,...,...,...,...,...,...,...,...,...,...
432841,432842,201 E 38Th St,NaN,Sioux Falls,SD,57105,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458,Street Address,SD
432842,432843,6700 E 41St St,NaN,Sioux Falls,SD,57110,NaN,"6700 E 41ST ST, SIOUX FALLS, SD 57110",43.516499,-96.642310,Street Address,SD
432843,432844,216 10Th St Se,NaN,Watertown,SD,57201,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432,Street Address,SD
432847,432848,1250 Vollintine Ave,NaN,Memphis,TN,38107,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851,Street Address,TN


In [18]:
OMOP_location_path = os.path.join(abs_path, 'output', 'OMOP_location.csv')

OMOP_location_copy.to_csv(OMOP_location_path, index=False)

# Data Quality Check
* address_1 should not be null and must be alphanumeric combination row-wise, may contain special characters
* address_2 can be null, otherwise alphanumeric combination
* city cannot have non-alphanumeric characters
* state must be 2-character abbreviation
* filter out non-street addresses by `address_type`

1) Parse with `usaddress` library
2) Data quality check above
3) Run custom parser function

* TO DO
  * Fix `West Virginia` state abbreviation

## Parse Failed Addresses with Custom Parser

In [20]:
# OMOP_location_path = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_location.csv'))
OMOP_location = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_location.csv'))
OMOP_location.shape

C:\Users\bchan\AppData\Local\Temp\ipykernel_13244\439158993.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  OMOP_location = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_location.csv'))


(386605, 12)

In [27]:
def parse_address(df,
                  address_col,
                  state_full_pattern,
                  state_abbr_pattern,
                  zip_code_pattern=r"[0-9]{5}(?:-[0-9]{4})?"
                  # state_code_pattern=r"[A-Z][a-z]+(?: +[A-Z][a-z]+)*)"
                 ):
    """
    Parse full address string to OMOP components by Regex search

    Parameters
    ----------
    df (DataFrame): Pandas DataFrame of source centroids with dates and user IDs

    Returns
    -------
    parse_df (DataFrame): DataFrame with parsed OMOP address components
    """
    tmp = []
    for i,row in df.iterrows():
        addr_components = row[address_col].split(',')

        # parse address if no RegEx match for 'APT'
        if len(re.findall(r'APT', row[address_col], flags=re.IGNORECASE)) == 0:
            state_zip = addr_components[2].split(' ')
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)   
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)
                
        # # parse address if RegEx match for 'APT' to address_1 & address_2
        elif len(re.findall(r'APT', row[address_col], flags=re.IGNORECASE)) > 0:
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', row[address_col], flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)
                    
        # # parse address if RegEx match for 'Suite' to to address_1 & address_2
        elif len(re.findall(r'SUITE', row[address_col], flags=re.IGNORECASE)) > 0:
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]

                tmp.append(row)

            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)        
        
    return pd.DataFrame(tmp)

In [28]:
# get failed addresses that do not have correct state abbreviation
OMOP_location['state_abbr'] = OMOP_location['state_abbr'].astype(str)
OMOP_state_failed = OMOP_location.loc[OMOP_location.state_abbr.str.len() > 2]
# state_condition = np.where(OMOP_location['state_abbr'].str.len() > 2)
# OMOP_state_failed = OMOP_location.loc[OMOP_location.state_abbr.map(lambda x: ]
OMOP_state_failed

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
220,222,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, KINGMAN, ME 04451",45.639023,-68.263641,NaN,nan
552,562,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, KINGFIELD, ME 04947",44.960341,-70.159052,NaN,nan
751,763,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, BROOKLINE, NH 03033",42.736111,-71.663431,NaN,nan
1057,1074,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, WONALANCET, NH 03897",43.908447,-71.344152,NaN,nan
1212,1232,Nan,NaN,Nan,NaN,NaN,NaN,"GENERAL DELIVERY, N HARTLAND, VT 05052",43.591020,-72.356611,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
386152,432158,Nan,NaN,Nan,NaN,NaN,NaN,"SAN VICENTE, SAIPAN, MP 96950",15.153098,145.739420,NaN,nan
386153,432166,Nan,NaN,Nan,NaN,NaN,NaN,"KOBLERVILLE, SAIPAN, MP 96950",15.120867,145.708247,NaN,nan
386154,432174,Nan,NaN,Nan,NaN,NaN,NaN,"CARR 687 KM 7 TORTUGUERO, VEGA BAJA, PR 00693",18.447033,-66.417684,NaN,nan
386155,432175,Nan,NaN,Nan,NaN,NaN,NaN,"CALLE TULIPAN RB20 URB. ROSALEDA II, TOA BAJA,...",18.445281,-66.172373,NaN,nan


In [29]:
%%time
# run custom parser on failed addresses above

state_full_pattern = r"/AL|Alabama|AK|Alaska|AZ|Arizona|AR|Arkansas|CA|California|CO|Colorado|CT|Connecticut|DE|Delaware|FL|Florida|GA|Georgia|HI|Hawaii|ID|Idaho|IL|Illinois|IN|Indiana|IA|Iowa|KS|Kansas|KY|Kentucky|LA|Louisiana|ME|Maine|MD|Maryland|MA|Massachusetts|MI|Michigan|MN|Minnesota|MS|Mississippi|MO|Missouri|MT|Montana|NE|Nebraska|NV|Nevada|NH|New Hampshire|NJ|New Jersey|NM|New Mexico|NY|New York|NC|North Carolina|ND|North Dakota|OH|Ohio|OK|Oklahoma|OR|Oregon|PA|Pennsylvania|RI|Rhode Island|SC|South Carolina|SD|South Dakota|TN|Tennessee|TX|Texas|UT|Utah|VT|Vermont|VA|Virginia|WA|Washington|WV|West Virginia|WI|Wisconsin|WY|Wyoming/"
state_abbr_pattern = re.compile(r'\b(AZ|CA|...|NJ|N\.J\.|NM|N\.M\.|...)\b')
state_abbr_case = r"^([Aa][LKSZRAEPlkszraep]|[Cc][AOTaot]|[Dd][ECec]|[Ff][LMlm]|[Gg][AUau]|[Hh][Ii]|[Ii][ADLNadln]|[Kk][SYsy]|[Ll][Aa]|[Mm][ADEHINOPSTadehinopst]|[Nn][CDEHJMVYcdehjmvy]|[Oo][HKRhkr]|[Pp][ARWarw]|[Rr][Ii]|[Ss][CDcd]|[Tt][NXnx]|[Uu][Tt]|[Vv][AITait]|[Ww][AIVYaivy])$"
zip_code_pattern=r"[0-9]{5}(?:-[0-9]{4})?"

new_df = parse_address(df=OMOP_state_failed,
                       address_col = 'location_source_value',
                       state_full_pattern=state_full_pattern, 
                       state_abbr_pattern=state_abbr_pattern)

new_df

CPU times: total: 109 ms
Wall time: 242 ms


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
220,222,LOCAL,NaN,KINGMAN,ME,04451,NaN,"LOCAL, KINGMAN, ME 04451",45.639023,-68.263641,NaN,nan
552,562,LOCAL,NaN,KINGFIELD,ME,04947,NaN,"LOCAL, KINGFIELD, ME 04947",44.960341,-70.159052,NaN,nan
751,763,LOCAL,NaN,BROOKLINE,NH,03033,NaN,"LOCAL, BROOKLINE, NH 03033",42.736111,-71.663431,NaN,nan
1057,1074,LOCAL,NaN,WONALANCET,NH,03897,NaN,"LOCAL, WONALANCET, NH 03897",43.908447,-71.344152,NaN,nan
1212,1232,GENERAL DELIVERY,NaN,N HARTLAND,VT,05052,NaN,"GENERAL DELIVERY, N HARTLAND, VT 05052",43.591020,-72.356611,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
386152,432158,SAN VICENTE,NaN,SAIPAN,MP,96950,NaN,"SAN VICENTE, SAIPAN, MP 96950",15.153098,145.739420,NaN,nan
386153,432166,KOBLERVILLE,NaN,SAIPAN,MP,96950,NaN,"KOBLERVILLE, SAIPAN, MP 96950",15.120867,145.708247,NaN,nan
386154,432174,CARR 687 KM 7 TORTUGUERO,NaN,VEGA BAJA,PR,00693,NaN,"CARR 687 KM 7 TORTUGUERO, VEGA BAJA, PR 00693",18.447033,-66.417684,NaN,nan
386155,432175,CALLE TULIPAN RB20 URB. ROSALEDA II,NaN,TOA BAJA,PR,00949,NaN,"CALLE TULIPAN RB20 URB. ROSALEDA II, TOA BAJA,...",18.445281,-66.172373,NaN,nan


In [30]:
# replace full state name to abbreviations for above addresses
new_df['state_abbr'] = new_df.state.apply(lambda x: multipleReplace(str(x).strip(), us_state_to_abbrev))
new_df

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
220,222,LOCAL,NaN,KINGMAN,ME,04451,NaN,"LOCAL, KINGMAN, ME 04451",45.639023,-68.263641,NaN,ME
552,562,LOCAL,NaN,KINGFIELD,ME,04947,NaN,"LOCAL, KINGFIELD, ME 04947",44.960341,-70.159052,NaN,ME
751,763,LOCAL,NaN,BROOKLINE,NH,03033,NaN,"LOCAL, BROOKLINE, NH 03033",42.736111,-71.663431,NaN,NH
1057,1074,LOCAL,NaN,WONALANCET,NH,03897,NaN,"LOCAL, WONALANCET, NH 03897",43.908447,-71.344152,NaN,NH
1212,1232,GENERAL DELIVERY,NaN,N HARTLAND,VT,05052,NaN,"GENERAL DELIVERY, N HARTLAND, VT 05052",43.591020,-72.356611,NaN,VT
...,...,...,...,...,...,...,...,...,...,...,...,...
386152,432158,SAN VICENTE,NaN,SAIPAN,MP,96950,NaN,"SAN VICENTE, SAIPAN, MP 96950",15.153098,145.739420,NaN,MP
386153,432166,KOBLERVILLE,NaN,SAIPAN,MP,96950,NaN,"KOBLERVILLE, SAIPAN, MP 96950",15.120867,145.708247,NaN,MP
386154,432174,CARR 687 KM 7 TORTUGUERO,NaN,VEGA BAJA,PR,00693,NaN,"CARR 687 KM 7 TORTUGUERO, VEGA BAJA, PR 00693",18.447033,-66.417684,NaN,PR
386155,432175,CALLE TULIPAN RB20 URB. ROSALEDA II,NaN,TOA BAJA,PR,00949,NaN,"CALLE TULIPAN RB20 URB. ROSALEDA II, TOA BAJA,...",18.445281,-66.172373,NaN,PR


In [31]:
# update OMOP_location with addresses parsed with custom parser
OMOP_location_updated = new_df.combine_first(OMOP_location)
OMOP_location_updated

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME
...,...,...,...,...,...,...,...,...,...,...,...,...
386600,432842,201 E 38Th St,NaN,Sioux Falls,SD,57105.0,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458,Street Address,SD
386601,432843,6700 E 41St St,NaN,Sioux Falls,SD,57110.0,NaN,"6700 E 41ST ST, SIOUX FALLS, SD 57110",43.516499,-96.642310,Street Address,SD
386602,432844,216 10Th St Se,NaN,Watertown,SD,57201.0,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432,Street Address,SD
386603,432848,1250 Vollintine Ave,NaN,Memphis,TN,38107.0,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851,Street Address,TN


In [10]:
def zip_leadingZeros(x):
    x.rjust(5, '0')

In [32]:
# pad leading zero to zipcode if not 5-digits
OMOP_location_updated['zip'] = OMOP_location_updated.zip.apply(lambda x: str(x).zfill(5))

# remove trailing zeros and decimal point
OMOP_location_updated['zip'] = OMOP_location_updated.zip.apply(lambda x: x.rstrip(".0") if ".0" in x else x)

OMOP_location_updated

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME
...,...,...,...,...,...,...,...,...,...,...,...,...
386600,432842,201 E 38Th St,NaN,Sioux Falls,SD,57105,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458,Street Address,SD
386601,432843,6700 E 41St St,NaN,Sioux Falls,SD,5711,NaN,"6700 E 41ST ST, SIOUX FALLS, SD 57110",43.516499,-96.642310,Street Address,SD
386602,432844,216 10Th St Se,NaN,Watertown,SD,57201,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432,Street Address,SD
386603,432848,1250 Vollintine Ave,NaN,Memphis,TN,38107,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851,Street Address,TN


## Classify Failures
* 0: SUCCESSFUL GEOCODE
* 1: FAILED GEOCODE (UNSPECIFIED)
* 2:  FAILED DUE TO PO BOX ADDRESS (CANNOT BE RECOVERED FROM FAILURE)
* 3:  FAILED GEOCODE (UNSPECIFIED) AND EXCLUDED DUE TO NON-WASHINGTON STATE AND NON-WASHINGTON ZIP CODE ENCODING
* 4:  FAILED DUE TO RETURNED / UNDELIVERABLE / HOMELESS ANNOTATION [CANNOT BE RECOVERED FROM FAILURE]
* 5:  FAILED DUE TO INCOMPLETE ADDRESS (ADDRESS LINES WERE EITHER ALL NUMBERS OR ALL LETTERS)  [CANNOT BE RECOVERED FROM FAILURE]
* 6:  FAILED BECAUSE STREET ADDRESS IN LINE1 IS FLIPPED WITH LINE 2 (e.g., LINE 1 is the apartment number and LINE 2 has street address) 
* 7:  FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS
* 8:  FAILED DUE TO ADDRESS LINES CONTAINING NAMES or MAILBOX NUMBERS

In [18]:
# IGNORE THIS CELL
def custom_flag(x):
    """
    Post-hoc data quality check of parsed addresses
    """  
    # if 'PO' in str(x.address_1) or 'P.O.' in str(x.address_1):
    # if 'PO' in str(x.address_1).strip() or 'P.O.' in str(x.address_1):
    if re.match('PO', x.address_1, re.IGNORECASE):
        return 'FAILED DUE TO PO BOX ADDRESS'
    elif re.match('APT', x.address_1, re.IGNORECASE) or re.match('SUITE', x.address_1, re.IGNORECASE):
        return 'FAILED DUE TO STREET ADDRESS IN LINE_1 IS FLIPPED WITH LINE_2'
    elif not x.address_1[0].isdigit():
        return 'FAILED DUE TO STREET ADDRESS STARTS WITH LETTER'
    # check address_1 only contains alphanumeric characters (spaces are ok)
    elif any(not c.isalnum() and not c.isspace() for c in x.address_1):
        return 'FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS'
    elif len(x.state_abbr) > 2:
        return 'FAILED DUE TO INCORRECT STATE FORMAT'
    elif x[['address_1', 'city', 'state', 'zip']].isnull().any():
        return 'FAILED DUE TO INCOMPLETE PARSING'
    else:
        return 'SUCCESSFUL ADDRESS'

In [54]:
%%time

# IGNORE THIS CELL
OMOP_location_updated['flag'] = OMOP_location_updated.apply(lambda x: custom_flag(x), axis=1)

OMOP_location_updated

CPU times: total: 33.7 s
Wall time: 40.3 s


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,523 E Broadway,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,454 Essex St,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,569 Broadway,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,3210 Southwestern Blvd,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,431 Campground Rd,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94671,104098,1300 Peachtree Industrial Boulevard,1300 Peachtree Industrial Boulevard,Suwanee,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS
94672,104099,2660 Satellite Boulevard Northwest,2660 Satellite Boulevard Northwest,Duluth,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS
94673,104100,3685 Braselton Highway,3685 Braselton Highway,Dacula,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS
94674,104101,1055 Dove Run Road,1055 Dove Run Road,Lexington,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS


In [150]:
# IGNORE THIS CELL
OMOP_location_updated.flag.value_counts()

flag
SUCCESSFUL ADDRESS                                 12879
FAILED DUE TO STREET ADDRESS STARTS WITH LETTER      821
FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS         588
FAILED DUE TO INCORRECT STATE FORMAT                  11
FAILED DUE TO PO BOX ADDRESS                          11
FAILED DUE TO INCOMPLETE PARSING                       1
Name: count, dtype: int64

In [48]:
OMOP_location_updated_path = os.path.join(abs_path, 'output', 'OMOP_location_flagged.csv')

OMOP_location_updated.to_csv(OMOP_location_updated_path, index=False)

In [33]:
# USE THIS TO FLAG ADDRESSES
def custom_flag(x):
    """
    Post-hoc data quality check of parsed addresses
    """  
    # check if PO box address
    if (re.match('PO BOX', x.address_1, re.IGNORECASE) or re.match('PO BOX', x.address_2, re.IGNORECASE))\
        or ('P.O' in str(x.address_1) or 'P.O' in str(x.address_2))\
        or ('P O' in str(x.address_1) or 'P O' in str(x.address_2))\
        or ('PSC' in str(x.address_1) or 'PSC' in str(x.address_2))\
        or ('PNB' in str(x.address_1) or 'PNB' in str(x.address_2))\
        or ('PMB' in str(x.address_1) or 'PMB' in str(x.address_2)):
        return 'FAILED DUE TO PO BOX ADDRESS'
    # check if street address starts with a non-digit character
    elif not x.address_1[0].isdigit() and not x.address_1.startswith("P"):
        return 'FAILED DUE TO STREET ADDRESS STARTS WITH LETTER'
    # check if address_2 and address_1 are flipped
    elif re.match('APT', x.address_1, re.IGNORECASE) or re.match('SUITE', x.address_1, re.IGNORECASE):
        return 'FAILED DUE TO STREET ADDRESS IN LINE_1 IS FLIPPED WITH LINE_2'
    # check address_1 only contains alphanumeric characters (spaces are ok)
    elif any(not c.isalnum() and not c.isspace() for c in x.address_1):
        return 'FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS'
    # check if parsed 'state' component matches a US state or territory abbreviation
    elif len(x.state_abbr) > 2:
        return 'FAILED DUE TO INCORRECT STATE FORMAT'
    # check if any of the required address components did not parse from the full address
    elif x[['address_1', 'city', 'state', 'zip']].isnull().any():
        return 'FAILED DUE TO INCOMPLETE PARSING'
    elif len(str(x.zip)) != 5:
        return 'FAILED DUE TO NON 5-DIGIT ZIPCODE'
    else:
        return 'SUCCESSFUL ADDRESS'

In [34]:
OMOP_location_updated['address_1'] = OMOP_location_updated['address_1'].astype(str)
OMOP_location_updated['address_2'] = OMOP_location_updated['address_2'].astype(str)

In [35]:
%%time
OMOP_location_updated['flag'] = OMOP_location_updated.apply(lambda x: custom_flag(x), axis=1)

OMOP_location_updated

CPU times: total: 1min 14s
Wall time: 2min 33s


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,nan,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,nan,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,nan,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,nan,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,nan,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386600,432842,201 E 38Th St,nan,Sioux Falls,SD,57105,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458,Street Address,SD,SUCCESSFUL ADDRESS
386601,432843,6700 E 41St St,nan,Sioux Falls,SD,5711,NaN,"6700 E 41ST ST, SIOUX FALLS, SD 57110",43.516499,-96.642310,Street Address,SD,FAILED DUE TO NON 5-DIGIT ZIPCODE
386602,432844,216 10Th St Se,nan,Watertown,SD,57201,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432,Street Address,SD,SUCCESSFUL ADDRESS
386603,432848,1250 Vollintine Ave,nan,Memphis,TN,38107,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851,Street Address,TN,SUCCESSFUL ADDRESS


In [36]:
OMOP_location_updated.flag.value_counts()

flag
SUCCESSFUL ADDRESS                                 272963
FAILED DUE TO PO BOX ADDRESS                        50985
FAILED DUE TO NON 5-DIGIT ZIPCODE                   38701
FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS        17539
FAILED DUE TO STREET ADDRESS STARTS WITH LETTER      6391
FAILED DUE TO INCORRECT STATE FORMAT                   26
Name: count, dtype: int64

In [37]:
# filter only 'SUCCESSFUL ADDRESS' flag
OMOP_location_updated_success = OMOP_location_updated.loc[OMOP_location_updated.flag == 'SUCCESSFUL ADDRESS']
OMOP_location_updated_success

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,nan,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,nan,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,nan,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,nan,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,nan,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386599,432840,200 Warrior Drive,nan,Elloree,SC,29047,NaN,"200 WARRIOR DRIVE, ELLOREE, SC 29047",33.529769,-80.551816,Street Address,SC,SUCCESSFUL ADDRESS
386600,432842,201 E 38Th St,nan,Sioux Falls,SD,57105,NaN,"201 E 38TH ST, SIOUX FALLS, SD 57105",43.517077,-96.724458,Street Address,SD,SUCCESSFUL ADDRESS
386602,432844,216 10Th St Se,nan,Watertown,SD,57201,NaN,"216 10TH ST SE, WATERTOWN, SD 57201",44.894168,-97.099432,Street Address,SD,SUCCESSFUL ADDRESS
386603,432848,1250 Vollintine Ave,nan,Memphis,TN,38107,NaN,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",35.165500,-90.014851,Street Address,TN,SUCCESSFUL ADDRESS


In [38]:
# filter only 'FAILED' addresses
OMOP_location_updated_failed = OMOP_location_updated.loc[OMOP_location_updated.flag != 'SUCCESSFUL ADDRESS']
OMOP_location_updated_failed

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
154,156,Po Box 102,nan,Deerfield St,NJ,08313,NaN,"PO BOX 102, DEERFIELD ST, NJ 08313",39.523911,-75.236086,PO Box,NJ,FAILED DUE TO PO BOX ADDRESS
155,157,Po Box 6149,nan,China Village,ME,04926,NaN,"PO BOX 6149, CHINA VILLAGE, ME 04926",44.481721,-69.516751,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
156,158,Po Box 555,nan,Northeast Hbr,ME,04662,NaN,"PO BOX 555, NORTHEAST HBR, ME 04662",44.294140,-68.290211,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
157,159,Po Box 514,nan,Bangor,ME,04402,NaN,"PO BOX 514, BANGOR, ME 04402",44.801671,-68.772141,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
158,160,Po Box 826,nan,Presque Isle,ME,04769,NaN,"PO BOX 826, PRESQUE ISLE, ME 04769",46.681235,-68.010188,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386580,432821,250 Winthrop Street,nan,Woonsocket,RI,2895,NaN,"250 WINTHROP STREET, WOONSOCKET, RI 02895",42.011648,-71.493423,Street Address,RI,FAILED DUE TO NON 5-DIGIT ZIPCODE
386581,432822,1835 Eden Terrace,nan,Rock Hill,SC,2973,NaN,"1835 EDEN TERRACE, ROCK HILL, SC 29730",34.959324,-81.005689,Street Address,SC,FAILED DUE TO NON 5-DIGIT ZIPCODE
386585,432826,8150 Warren H. Abernathy Highway,nan,Spartanburg,SC,29301,NaN,"8150 WARREN H. ABERNATHY HIGHWAY, SPARTANBURG,...",34.936208,-82.006023,Street Address,SC,FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS
386591,432832,500 R.M. Foster Drive,nan,Cordova,SC,29039,NaN,"500 R.M. FOSTER DRIVE, CORDOVA, SC 29039",33.444233,-80.937657,Street Address,SC,FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS


In [39]:
# check PO box addresses
OMOP_location_updated.loc[OMOP_location_updated.flag == 'FAILED DUE TO PO BOX ADDRESS']

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
154,156,Po Box 102,nan,Deerfield St,NJ,08313,NaN,"PO BOX 102, DEERFIELD ST, NJ 08313",39.523911,-75.236086,PO Box,NJ,FAILED DUE TO PO BOX ADDRESS
155,157,Po Box 6149,nan,China Village,ME,04926,NaN,"PO BOX 6149, CHINA VILLAGE, ME 04926",44.481721,-69.516751,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
156,158,Po Box 555,nan,Northeast Hbr,ME,04662,NaN,"PO BOX 555, NORTHEAST HBR, ME 04662",44.294140,-68.290211,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
157,159,Po Box 514,nan,Bangor,ME,04402,NaN,"PO BOX 514, BANGOR, ME 04402",44.801671,-68.772141,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
158,160,Po Box 826,nan,Presque Isle,ME,04769,NaN,"PO BOX 826, PRESQUE ISLE, ME 04769",46.681235,-68.010188,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
382927,427823,Po Box 68,nan,Burlington,IL,60109,NaN,"PO BOX 68, BURLINGTON, IL 60109",42.018006,-88.485019,PO Box,IL,FAILED DUE TO PO BOX ADDRESS
384866,430405,Po Box 326,nan,Manlius,IL,61338,NaN,"PO BOX 326, MANLIUS, IL 61338",41.454928,-89.678301,PO Box,IL,FAILED DUE TO PO BOX ADDRESS
384870,430416,Po Box 570,nan,Bourbonnais,IL,60914,NaN,"PO BOX 570, BOURBONNAIS, IL 60914",41.177927,-87.892895,PO Box,IL,FAILED DUE TO PO BOX ADDRESS
386466,432645,Po Box 2811,nan,Mccall,ID,83638,NaN,"PO BOX 2811, MCCALL, ID 83638",44.907803,-116.098682,PO Box,ID,FAILED DUE TO PO BOX ADDRESS


In [40]:
# check PO box addresses
OMOP_location_updated.loc[OMOP_location_updated.flag == 'FAILED DUE TO NON 5-DIGIT ZIPCODE']

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
19777,24128,810 Old Salem Road,nan,Murfreesboro,TN,37129-4943,NaN,"810 Old Salem Road, Murfreesboro, TN 37129-4943",35.835119,-86.404821,Street Address,TN,FAILED DUE TO NON 5-DIGIT ZIPCODE
19916,24267,907 Murfreesboro Road,nan,Franklin,TN,37064-3002,NaN,"907 Murfreesboro Road, Franklin, TN 37064-3002",35.917312,-86.842461,Street Address,TN,FAILED DUE TO NON 5-DIGIT ZIPCODE
20094,24445,215 Noah Drive,nan,Franklin,TN,37064-3938,NaN,"215 Noah Drive, Franklin, TN 37064-3938",35.894806,-86.867364,Street Address,TN,FAILED DUE TO NON 5-DIGIT ZIPCODE
20288,24639,1504 West College Street,nan,Murfreesboro,TN,37129-1726,NaN,"1504 West College Street, Murfreesboro, TN 371...",35.862377,-86.412735,Street Address,TN,FAILED DUE TO NON 5-DIGIT ZIPCODE
20383,24734,1061 Cool Springs Boulevard,nan,Franklin,TN,37067-2718,NaN,"1061 Cool Springs Boulevard, Franklin, TN 3706...",35.933573,-86.794985,Street Address,TN,FAILED DUE TO NON 5-DIGIT ZIPCODE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386561,432787,3711 Clifton Ave,nan,Cincinnati,OH,4522,NaN,"3711 CLIFTON AVE, CINCINNATI, OH 45220",39.150291,-84.519218,Street Address,OH,FAILED DUE TO NON 5-DIGIT ZIPCODE
386575,432813,69 Meadowbrook Ave,nan,Hatboro,PA,1904,NaN,"69 MEADOWBROOK AVE, HATBORO, PA 19040",40.184631,-75.099371,Street Address,PA,FAILED DUE TO NON 5-DIGIT ZIPCODE
386580,432821,250 Winthrop Street,nan,Woonsocket,RI,2895,NaN,"250 WINTHROP STREET, WOONSOCKET, RI 02895",42.011648,-71.493423,Street Address,RI,FAILED DUE TO NON 5-DIGIT ZIPCODE
386581,432822,1835 Eden Terrace,nan,Rock Hill,SC,2973,NaN,"1835 EDEN TERRACE, ROCK HILL, SC 29730",34.959324,-81.005689,Street Address,SC,FAILED DUE TO NON 5-DIGIT ZIPCODE


In [41]:
OMOP_location_updated_path = os.path.join(abs_path, 'output', 'OMOP_location_flagged_successful.csv')
OMOP_location_updated_success.to_csv(OMOP_location_updated_path, index=False)

In [42]:
OMOP_location_failed_path = os.path.join(abs_path, 'output', 'OMOP_location_flagged_failed.csv')
OMOP_location_updated_failed.to_csv(OMOP_location_failed_path, index=False)

In [20]:
OMOP_location_updated_success = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_location_flagged_successful.csv'))
OMOP_location_updated_success

C:\Users\bchan\AppData\Local\Temp\ipykernel_200\1298676597.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  OMOP_location_updated_success = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_location_flagged_successful.csv'))


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78958,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS
78959,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS
78960,104100,3685 Braselton Highway,NaN,Dacula,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS
78961,104101,1055 Dove Run Road,NaN,Lexington,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS


In [31]:
pd.DataFrame(OMOP_location_updated_success.state_abbr.sort_values().value_counts())

,count
state_abbr,
SD,8401
CA,6526
CO,6029
AZ,5495
OR,4289
NE,3384
UT,2758
NM,2396
FL,2363


# Random Sample by US State and Territories

In [27]:
# drop US territories that don't have at least 10 addresses
territories_drop = ['MP', 'VI', 'PW', 'Of', 'PR', 'GU']

dropped_territories = OMOP_location_updated_success.loc[OMOP_location_updated_success.state_abbr.isin(territories_drop)]
OMOP_location_updated_success_drop = OMOP_location_updated_success.loc[~OMOP_location_updated_success.state_abbr.isin(territories_drop)]

print(OMOP_location_updated_success_drop.shape)

(12859, 13)


In [28]:
# groupby state and randomly sample 10 addresses from each
df_sample = OMOP_location_updated_success_drop.groupby('state_abbr').apply(lambda x: x.sample(n=10)).reset_index(drop = True)
df_sample

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,9981,339 East Dogwood Avenue,NaN,Palmer,AK,99645,NaN,"339 EAST DOGWOOD AVENUE, PALMER, AK 99645",61.602250,-149.110053,Street Address,AK,SUCCESSFUL ADDRESS
1,940,114 Illinois St,NaN,Anaktuvuk Pass,AK,99721,NaN,"114 ILLINOIS ST, ANAKTUVUK PASS, AK 99721",68.139836,-151.738557,Street Address,AK,SUCCESSFUL ADDRESS
2,18472,7801 E 32Nd Ave,NaN,Anchorage,AK,99504,NaN,"7801 E 32ND AVE, ANCHORAGE, AK 99504",61.192277,-149.735520,Street Address,AK,SUCCESSFUL ADDRESS
3,12642,950 East Bogard Road,NaN,Wasilla,AK,99654,NaN,"950 EAST BOGARD ROAD, WASILLA, AK 99654",61.587019,-149.424305,Street Address,AK,SUCCESSFUL ADDRESS
4,15961,12350 Industry Way,NaN,Anchorage,AK,99515,NaN,"12350 INDUSTRY WAY, ANCHORAGE, AK 99515",61.109120,-149.862291,Street Address,AK,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,13819,13794 Prairie Center Circle,NaN,Cheyenne,WY,82009,NaN,"13794 PRAIRIE CENTER CIRCLE, CHEYENNE, WY 82009",41.149766,-104.646764,Street Address,WY,SUCCESSFUL ADDRESS
506,14191,1550 United States Highway 20 South,NaN,Worland,WY,82401,NaN,"1550 UNITED STATES HIGHWAY 20 SOUTH, WORLAND, ...",43.956469,-108.020552,Street Address,WY,SUCCESSFUL ADDRESS
507,18197,445 South Main St,NaN,Lusk,Wyoming,82225,NaN,"445 South Main St, Lusk, Wyoming 82225",42.760784,-104.452038,Street Address,WY,SUCCESSFUL ADDRESS
508,5281,525 East Birch Street,NaN,Glenrock,WY,82637,NaN,"525 EAST BIRCH STREET, GLENROCK, WY 82637",42.861038,-105.857974,Street Address,WY,SUCCESSFUL ADDRESS


In [29]:
# concatenate dropped US territories to the other random samples
OMOP_sample = pd.concat([dropped_territories, df_sample])
OMOP_sample_path = os.path.join(abs_path, 'output', 'OMOP_sample.csv')

OMOP_sample.to_csv(OMOP_sample_path, index=False)

## Failed Address Sample

In [27]:
# drop US territories that don't have at least 10 addresses
territories_drop = ['MP', 'VI', 'PW', 'Of', 'PR', 'GU', 'AS', 'FM', 'MH']

dropped_territories = OMOP_location_updated_failed.loc[OMOP_location_updated_failed.state_abbr.isin(territories_drop)]
OMOP_location_updated_failed_drop = OMOP_location_updated_failed.loc[~OMOP_location_updated_failed.state_abbr.isin(territories_drop)]

print(OMOP_location_updated_failed_drop.shape)

(15702, 13)


In [28]:
# groupby state and randomly sample 10 addresses from each
df_sample = OMOP_location_updated_failed_drop.groupby('state_abbr').apply(lambda x: x.sample(n=10)).reset_index(drop = True)
df_sample

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,25327,Po Box 233498,nan,Anchorage,AK,99523,NaN,"PO BOX 233498, ANCHORAGE, AK 99523",61.168120,-149.837850,PO Box,AK,FAILED DUE TO STREET ADDRESS STARTS WITH LETTER
1,11160,Po Box 10878,nan,Fairbanks,AK,99710,NaN,"PO BOX 10878, FAIRBANKS, AK 99710",64.845089,-147.722031,PO Box,AK,FAILED DUE TO STREET ADDRESS STARTS WITH LETTER
2,25156,Po Box 43,nan,Eagle,AK,99738,NaN,"PO BOX 43, EAGLE, AK 99738",64.787199,-141.202161,PO Box,AK,FAILED DUE TO STREET ADDRESS STARTS WITH LETTER
3,24946,Po Box 85,nan,Petersburg,AK,99833,NaN,"PO BOX 85, PETERSBURG, AK 99833",56.810621,-132.945116,PO Box,AK,FAILED DUE TO STREET ADDRESS STARTS WITH LETTER
4,25400,Po Box 1078,nan,Kodiak,AK,99615,NaN,"PO BOX 1078, KODIAK, AK 99615",57.796287,-152.390811,PO Box,AK,FAILED DUE TO STREET ADDRESS STARTS WITH LETTER
...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,37750,2 W. Main Street,nan,Wardensville,West Virginia,26851,NaN,"2 W. Main Street, Wardensville, West Virginia ...",39.076470,-78.594965,Street Address,West VA,FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS
526,36423,103 Dodd Street,nan,Middlebourne,West Virginia,26149,NaN,"103 Dodd Street, Middlebourne, West Virginia 2...",39.564169,-80.995953,Street Address,West VA,FAILED DUE TO INCORRECT STATE FORMAT
527,34305,5229 Coal Heritage Road,nan,Iaeger,West Virginia,24844,NaN,"5229 Coal Heritage Road, Iaeger, West Virginia...",37.467707,-81.809246,Street Address,West VA,FAILED DUE TO INCORRECT STATE FORMAT
528,34301,160 Morgantown St,nan,Bruceton Mills,West Virginia,26525,NaN,"160 Morgantown St, Bruceton Mills, West Virgin...",39.658873,-79.639849,Street Address,West VA,FAILED DUE TO INCORRECT STATE FORMAT


In [29]:
# concatenate dropped US territories to the other random samples
OMOP_sample = pd.concat([dropped_territories, df_sample])
OMOP_sample_path = os.path.join(abs_path, 'output', 'OMOP_failed_sample.csv')

OMOP_sample.to_csv(OMOP_sample_path, index=False)

# Text Similarity Score
* Cosine similarity

In [3]:
nlp = spacy.load("en_core_web_lg")

# address strings
location_source_address = "525 EAST BIRCH STREET, GLENROCK, WY 82637"
parsed_address = "525 East Birch Street, Glenrock, WY 82637"

doc1 = nlp(location_source_address.title())
doc2 = nlp(parsed_address)

print(f"The similarity between \033[1m{location_source_address} \033[0mand \033[1m{parsed_address} is \033[0m{doc1.similarity(doc2)}.")

The similarity between 525 EAST BIRCH STREET, GLENROCK, WY 82637 and 525 East Birch Street, Glenrock, WY 82637 is 0.9866509809411516.


In [4]:
# IGNORE THIS !!!
nlp = spacy.load("en_core_web_lg")

# address strings
location_source_address = "525 EAST BIRCH STREET, GLENROCK, WY 82637"
parsed_address = "525 East Birch Street, Glenrock, WY 82637"

doc1 = nlp(location_source_address)
doc2 = nlp(parsed_address)

print(f"The similarity between \033[1m{location_source_address} \033[0mand \033[1m{parsed_address} is \033[0m{doc1.similarity(doc2)}.")

The similarity between 525 EAST BIRCH STREET, GLENROCK, WY 82637 and 525 East Birch Street, Glenrock, WY 82637 is 0.8022178579015921.


In [ ]:
nlp = spacy.load("en_core_web_lg")

def text_similarity(source_address, parsed_address):
    """
    Calculate cosine similarity between two strings

    Parameters
    ----------
    source_address (str): original address string input to a geocoder
    parsed_addres (str): returned address string from a geocoder

    Returns
    -------
    score (float)
    """
    # capitalize first letter only of each word
    doc1 = nlp(source_address.title())
    doc2 = nlp(parsed_address.title())

    print(f"The similarity between \033[1m{source_address} \033[0mand \033[1m{parsed_address} is \033[0m{doc1.similarity(doc2)}.")
    return doc1.similarity(doc2)